In [65]:
import matplotlib
matplotlib.use('Agg')  # Backend non-interactif

import matplotlib.pyplot as plt
import os
import glob
import numpy as np
import astropy.units as u
import random

from astropy.wcs import WCS
from textwrap import fill
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
from astropy.io import fits
from astropy.coordinates import search_around_sky
from astropy.visualization import simple_norm
from astropy.table import join, hstack

from constantes import LIM_FLUX_CLUSTER, LIM_FLUX_AGN, SEARCH_RADIUS_CLUSTER, SEARCH_RADIUS_AGN, EXT_LIKE_C1, EXT_LIKE_C2, EXT_C1_C2, PNT_DET_ML_SPURIOUS, EXT_LIKE_SPURIOUS
from constantes import print_parameters

print_parameters()

╭─────────────────────────────────────────────╮
│                  PARAMÈTRES                 │
├─────────────────────────────────────────────┤
│ LIM_FLUX_CLUSTER      : 1.00e-15 erg/cm²/s 
│ LIM_FLUX_AGN          : 1.00e-15 erg/cm²/s 
│ SEARCH_RADIUS_CLUSTER : 30.00    arcsec    
│ SEARCH_RADIUS_AGN     : 10.00    arcsec    
│ EXT_LIKE_C1           : 33       
│ EXT_LIKE_C2           : 15       
│ EXT_C1_C2             : 5        
│ window_size           : 7.0      arcmin    
│ PNT_DET_ML_SPURIOUS   : 20       
│ EXT_LIKE_SPURIOUS     : 15       
╰─────────────────────────────────────────────╯


In [66]:
merging = False

if(merging):
    input_dir = "/local/home/sh275430/Documents/TransformerProject/data/catalogs/Xamin_stack_fxabs2e-16"
    output_path = os.path.join(input_dir, "merged_catalog.fits")  # Même dossier que les fichiers sources

    try:
        # Lister les fichiers FITS (exclure le fichier de sortie s'il existe)
        fits_files = [f for f in glob.glob(os.path.join(input_dir, "*.fits")) 
                    if not os.path.basename(f) == "merged_catalog.fits"]
        
        if not fits_files:
            raise ValueError("Aucun fichier FITS trouvé dans le dossier spécifié")

        tables = []
        current_max_id = 0  # Pour garantir des new_ID uniques
        
        for file_path in sorted(fits_files):
            with fits.open(file_path) as hdul:
                data = Table(hdul[1].data)
                
                # Création des new_ID uniques séquentiels
                num_sources = len(data)
                data['ID_Xamin'] = np.arange(current_max_id + 1, current_max_id + 1 + num_sources)
                current_max_id += num_sources
                
                tables.append(data)

        merged_table = vstack(tables)
        
        if os.path.exists(output_path):
            os.remove(output_path)  # Suppression explicite de l'ancienne version
        merged_table.write(output_path, overwrite=True)
        
        # Vérification finale
        if os.path.exists(output_path):
            print(f"Catalogues fusionnés sauvegardés dans {output_path}")
            print(f"Nombre total de sources : {len(merged_table)}")
        else:
            raise RuntimeError("Échec de l'écriture du fichier")

    except Exception as e:
        print(f"ERREUR : {str(e)}")
        # Nettoyage en cas d'échec
        if 'merged_table' in locals() and os.path.exists(output_path):
            os.remove(output_path)

In [67]:
catalog_path_aftXamin = os.path.expanduser('~/Documents/TransformerProject/data/catalogs/Xamin_stack_fxabs2e-16/merged_catalog.fits')
data_Xamin = Table.read(catalog_path_aftXamin)
data_Xamin['new_ID'] = np.arange(len(data_Xamin))

In [68]:
RADIUS_XAMIN = 4  #arcsec

In [69]:
coords = SkyCoord(ra=data_Xamin['PNT_RA'], dec=data_Xamin['PNT_DEC'], unit='deg')

#Calcul de ttes les paires a moins de RADIUS_XAMIN
idx1, idx2, sep2d, _ = coords.search_around_sky(coords, seplimit= RADIUS_XAMIN* u.arcsec)

#Filtrage des paires distinctes
mask = idx1 < idx2
idx1 = idx1[mask]
idx2 = idx2[mask]

to_remove = set()
for i, j in zip(idx1, idx2):
    id_i = data_Xamin['new_ID'][i]
    id_j = data_Xamin['new_ID'][j]
    to_remove.add(i if id_i > id_j else j)

#Suppression des lignes
mask_keep = np.ones(len(data_Xamin), dtype = bool)
mask_keep[list(to_remove)] = False
data_Xamin_cleaned = data_Xamin[mask_keep]

data_Xamin_cleaned.remove_column('new_ID')

In [70]:
data_Xamin_cleaned

BOX_ID_SRC,INST0,BAND0,INST1,BAND1,INST0_EXP,INST1_EXP,INST0_GAPFLAG,INST1_GAPFLAG,GAP_NEIGHBOUR,INPUT_X_IMA,INPUT_Y_IMA,PNT_DET_ML,EXT_DET_ML,EPN_DET_ML,DBL_DET_ML,EPN_ML_EXT,PNT_X_IMA,PNT_X_IMA_ERR,PNT_Y_IMA,PNT_IMA_ERR,PNT_RA,PNT_DEC,PNT_RADEC_ERR,PNT_RATE_MOS,PNT_RATE_MOS_ERR,PNT_RATE_PN,PNT_RATE_PN_ERR,PNT_SCTS_MOS,PNT_SCTS_MOS_ERR,PNT_SCTS_PN,PNT_SCTS_PN_ERR,PNT_BG_MAP_MOS,PNT_BG_MAP_PN,PNT_PIX_DEV,PNT_N_ITER,PNT_CUTRAD,EXT_LIKE,EXT,EXT_ERR,EXT_X_IMA,EXT_X_IMA_ERR,EXT_Y_IMA,EXT_Y_IMA_ERR,EXT_RA,EXT_DEC,EXT_RADEC_ERR,EXT_RATE_MOS,EXT_RATE_MOS_ERR,EXT_RATE_PN,EXT_RATE_PN_ERR,EXT_SCTS_MOS,EXT_SCTS_MOS_ERR,EXT_SCTS_PN,EXT_SCTS_PN_ERR,EXT_BG_MAP_MOS,EXT_BG_MAP_PN,EXT_PIX_DEV,EXT_N_ITER,EXT_CUTRAD,EPN_LIKE_EXT,EPN_LIKE_PNT,EPN_EXT,EPN_EXT_ERR,EPN_RATIO,EPN_RATIO_ERR,EPN_X_IMA,EPN_X_IMA_ERR,EPN_Y_IMA,EPN_Y_IMA_ERR,EPN_RA,EPN_DEC,EPN_RADEC_ERR,EPN_RATE_MOS,EPN_RATE_MOS_ERR,EPN_RATE_PN,EPN_RATE_PN_ERR,EPN_SCTS_MOS,EPN_SCTS_MOS_ERR,EPN_SCTS_PN,EPN_SCTS_PN_ERR,EPN_BG_MAP_MOS,EPN_BG_MAP_PN,EPN_PIX_DEV,EPN_N_ITER,EPN_CUTRAD,DBL_LIKE,DBL_SEP,DBL_SEP_ERR,DBL_RATIO,DBL_RATIO_ERR,DBL_THETA,DBL_X_IMA,DBL_X_IMA_ERR,DBL_Y_IMA,DBL_Y_IMA_ERR,DBL_RA,DBL_DEC,DBL_RADEC_ERR,DBL_RATE_MOS,DBL_RATE_MOS_ERR,DBL_RATE_PN,DBL_RATE_PN_ERR,DBL_SCTS_MOS,DBL_SCTS_MOS_ERR,DBL_SCTS_PN,DBL_SCTS_PN_ERR,DBL_BG_MAP_MOS,DBL_BG_MAP_PN,DBL_PIX_DEV,DBL_N_ITER,DBL_CUTRAD,ID_Xamin
int32,bytes8,bytes8,bytes8,bytes8,float64,float64,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,int64
1,MOS,b2,PN,b2,72641.04625,45652.07223232582,6,88,1,1391.999999999969,131.9999999999966,130.8668216013175,673.4227632936643,420.5005864653804,0.0,-256.3711862861819,1391.999999999969,--,131.9999999999966,--,147.15007880983848,1.274979808591081,--,0.0011664784168358307,--,0.0038945989330756853,--,80.46353176834805,--,110.96143519016371,--,0.11126679386477972,0.3178819882451341,0.0,49,103.40306854248047,552.0630426876887,60.945570407801625,--,1403.5064201311498,--,134.9638448605815,--,147.15807113383846,1.2770365873212615,--,0.03198573746013102,--,0.10724625471944374,--,1090.2261404939334,--,1616.2999410757145,--,0.05605396854873696,0.1718298746227782,3.5996670575460574,306,103.40306854248047,0.0,295.6918564015068,15.492683800885771,--,3.453013885436904e-06,--,1391.999999999969,--,131.9999999999966,--,147.15007880983848,1.274979808591081,--,0.006525849799234403,--,0.00916481518247098,--,442.5378604535034,--,260.761977933242,--,0.09109371120796768,0.30494126850982606,0.0,263,103.40306854248047,0.0,12.925383567810059,--,1.1,--,8.274860119995813e-10,1403.330076536074,--,131.36850353764828,--,147.15794810998386,1.2745400762580104,--,0.027380781979145955,--,0.055515603646996314,--,1885.3808001476937,--,1585.3514973243953,--,0.00982970645900138,0.2016219314876888,0.0,0,103.40306854248047,1
2,MOS,b2,PN,b2,71150.000625,35575.0003125,14,48,1,1612.0000000000032,41.999999999982606,2232.4720611063603,2216.1611567352415,2176.243271520619,2278.9281696274984,-43.50744779618617,1612.0000000000032,--,41.999999999982606,--,147.30286009394996,1.2124593369471506,--,0.007751226492524443,--,0.008107717568555581,--,545.0971489383618,--,284.36827633781604,--,0.07539847807505162,0.06891482373131319,0.0,35,56.552799224853516,0.0,0.3839290083349125,--,1612

In [71]:
print(f"Nombre de sources Xamin avant la suppresion des doublons: {len(data_Xamin)}")
print(f"Nombre de sources Xamin apres la suppresion des doublons: {len(data_Xamin_cleaned)}")

Nombre de sources Xamin avant la suppresion des doublons: 30279
Nombre de sources Xamin apres la suppresion des doublons: 25319


In [72]:
output_path = os.path.expanduser(f'~/Documents/TransformerProject/data/catalogs/Xamin_stack_fxabs2e-16/merged_catalog_cleaned.fits')
data_Xamin_cleaned.write(output_path, format='fits', overwrite=True)

print(f"Catalogue Xamin sauvegardé dans : {output_path}")

Catalogue Xamin sauvegardé dans : /local/home/sh275430/Documents/TransformerProject/data/catalogs/Xamin_stack_fxabs2e-16/merged_catalog_cleaned.fits
